In [2]:
# from wordcloud import WordCloud
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

 
path = "C:\\workspace\\특화프로젝트\\chromedriver-win64\\chromedriver.exe" # 웹드라이버 실행
 
driver = webdriver.Chrome() # 드라이버 경로 설정
url_list = [] # url을 저장하기 위한 변수
content_list = []

cat_values = ["111", "115", "116"]

def collect_urls_for_cat(cat):
    for i in range(1, 11):  
        url = f'https://kid.chosun.com/list_kj.html?catid={cat}&pn={i}'
        driver.get(url)
        time.sleep(0.5)

        for j in range(1, 11): 
            try:
                title_element = driver.find_element(By.XPATH, f'//*[@id="container"]/section[2]/article/ul/li[{j}]/div[2]/div[1]/a ')
                title_url = title_element.get_attribute('href')
                url_list.append(title_url)
            except Exception as e:
                print(f"Error finding element: {e}")  
                       
        
for cat in cat_values:
    collect_urls_for_cat(cat)
        
print("url 수집 끝")
driver.close()

print("수집한 url 개수 : "+str(len(url_list)))

url 수집 끝
수집한 url 개수 : 300


In [4]:
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
df = pd.DataFrame(columns=['URL', 'title', 'content']) # content를 누적하기 위한 변수

for index, url in enumerate(url_list):
    level_text = ""
    title_text = ""
    content_text = ""
    question_text = ""
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    title = soup.find('div', class_='page_title_subject')
    if title:
        title_text = title.get_text(strip=True)
    else:
        title_text = "No Title Found"
        
    content_divs = soup.find_all('div', class_='Paragraph')
    for div in content_divs:
        paragraphs = div.find_all('p')
        for paragraph in paragraphs:
            content_text += paragraph.get_text().strip() + "\n"
        content_text += div.get_text().strip() + "\n"

    # Append the extracted content to the list
    content_list.append({
        'URL': url,
        'title': title_text,
        'content': content_text
    })
    
    if (index + 1) % 50 == 0:
        print(f"Processed {index + 1} URLs")

# Convert to a DataFrame for easier handling
df = pd.DataFrame(content_list)
print(df)

driver.quit()

Processed 50 URLs
Processed 100 URLs
Processed 150 URLs
Processed 200 URLs
Processed 250 URLs
Processed 300 URLs
                                                   URL  \
0    https://kid.chosun.com/site/data/html_dir/2024...   
1    https://kid.chosun.com/site/data/html_dir/2024...   
2    https://kid.chosun.com/site/data/html_dir/2024...   
3    https://kid.chosun.com/site/data/html_dir/2024...   
4    https://kid.chosun.com/site/data/html_dir/2024...   
..                                                 ...   
295  https://kid.chosun.com/site/data/html_dir/2022...   
296  https://kid.chosun.com/site/data/html_dir/2022...   
297  https://kid.chosun.com/site/data/html_dir/2022...   
298  https://kid.chosun.com/site/data/html_dir/2022...   
299  https://kid.chosun.com/site/data/html_dir/2022...   

                                                 title  \
0                [초·저 / 동식물 이야기] 비단뱀, 큰 먹이 삼키면 심장 커진대요   
1    [초·고 / 지구를 지켜줘] 중국이 양쯔강 싼샤댐 방류했더니 제주도 물고기가 다른 ...   
2        [초·고 / 

In [5]:
file_path = "data/raw/news/news_chosun_crawled.csv"
df.to_csv(file_path, index=False, encoding='utf-8-sig')

print("Data saved")

Data saved
